Motivation: In this notebook, I give an example of how to use my implementation of fast Kronecker inference for GPs with arbitrary likelihoods. See Flaxman et al (2015) "Fast Kronecker Inference in Gaussian Processes with non-Gaussian Likelihoods" for reference.

In [1]:
from kronecker import KroneckerSolver
import kernels as kern
from likelihoods import PoissonLike, GaussianLike
import data_utils as sim
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
from IPython.display import display
init_notebook_mode(connected=True)
from edward import rbf
import GPy
import numpy as np
import itertools
from kernels import RBF
from grid_utils import fill_grid
import sys
import tensorflow as tf
import tensorflow.contrib.eager as tfe
tfe.enable_eager_execution()
import pandas as pd

## Simulate some data

First, let's simulate some data on a 2D grid. Given a set of points $\{x_i\}_{i=1}^N$, we generate data via the following model:

$$f \sim ~\mathcal{GP}(\mu(x), K(x, x))$$

$$ y(x_i) \sim ~ \text{Poisson}(f(x_i))$$


Here's what the $x_i$'s look like:

In [2]:
X = sim.sim_X_equispaced(D = 2, N_dim = 30)

iplot([go.Scatter(x = X[:,0], y = X[:,1], mode = 'markers', marker=dict(size = 3,))])

Now we draw function values f from a GP with an RBF Kernel, and draw y based on f:

In [3]:
f = sim.sim_f(X, k=RBF(variance=1.0, length_scale=30.))
y = sim.poisson_draw(f, 0.5)
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=y, mode = 'markers', marker=dict(size = 2,))])

data_utils.py:23: RuntimeWarning:

covariance is not positive-semidefinite.



In [4]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z=f, mode = 'markers', marker=dict(size = 2,))])

## Inference

We're interested in the following: given y and x, can we infer the function values f(x)? We've made an assumption with the Poisson likelihood here, but this implementation should work with any differentiable likelihood.

Construct a KroneckerSolver object (given a kernel), and run the inference. This should converge within a few Newton iterations.

In [5]:
ks = KroneckerSolver(tf.ones([X.shape[0]], tf.float32)*np.mean(np.log(y)), RBF(variance=1.0, length_scale=20.0),
                     PoissonLike(), X, tfe.Variable(y, dtype = tf.float32), tau = 0.5, verbose = True)

ks.run(10)

Iteration:  <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>
 psi:  tf.Tensor(-483036.0, shape=(), dtype=float32)
step 0.5

Iteration:  tf.Tensor(1, shape=(), dtype=int32)
 psi:  tf.Tensor(-524998.0, shape=(), dtype=float32)
step 2.0

Iteration:  tf.Tensor(2, shape=(), dtype=int32)
 psi:  tf.Tensor(-529573.0, shape=(), dtype=float32)
step 1.0

Iteration:  tf.Tensor(3, shape=(), dtype=int32)
 psi:  tf.Tensor(-531419.0, shape=(), dtype=float32)
step 2.0

Iteration:  tf.Tensor(4, shape=(), dtype=int32)
 psi:  tf.Tensor(-531423.0, shape=(), dtype=float32)
step 1.0

Iteration:  tf.Tensor(5, shape=(), dtype=int32)
 psi:  tf.Tensor(-531444.0, shape=(), dtype=float32)
step 0.125

Iteration:  tf.Tensor(6, shape=(), dtype=int32)
 psi:  tf.Tensor(-531444.0, shape=(), dtype=float32)
step 0.0



(10, <tf.Tensor: id=201689, shape=(), dtype=int32, numpy=7>, 0.0)

Plot the inferred function values

In [6]:
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z= np.array(ks.f), mode = 'markers', marker=dict(size = 2,))])

In [7]:
var = ks.variance(20)
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z= var.numpy(), mode = 'markers', marker=dict(size = 2,))])

Inference on a partial grid

In [8]:
indices = np.sort(np.random.choice(X.shape[0], int(X.shape[0]*.15), replace = False))
X_partial = X[indices]
y_partial = y[indices]
X_partial = X_partial[np.lexsort((X_partial[:,1], X_partial[:,0]))]
X_part_tf = tf.constant(X_partial)
iplot([go.Scatter(x = X_partial[:,0], y = X_partial[:,1], mode = 'markers', marker=dict(size = 5,))])

In [9]:
X_full, y_full, obs_idx, imag_idx = fill_grid(X_partial, y_partial)

y_tf = tfe.Variable(y_full[obs_idx], dtype = tf.float32)
mu = tf.ones([X_full.shape[0]], tf.float32)*np.mean(np.log(y_full[obs_idx]))

In [10]:
ks = KroneckerSolver(mu, RBF(variance=1.0, length_scale=20.0),
                     PoissonLike(), X_full, y_tf, obs_idx=obs_idx, verbose = True)

In [11]:
y_full.shape

(900,)

In [12]:
print "running"
ks.run(20)
iplot([go.Scatter3d(x = X_full[:, 0], y = X_full[:, 1], z= np.array(ks.f_pred), mode = 'markers', marker=dict(size = 2))])

running
Iteration:  <tf.Variable 'Variable:0' shape=() dtype=int32, numpy=0>
 psi:  tf.Tensor(-78926.9, shape=(), dtype=float32)
step 0.5

Iteration:  tf.Tensor(1, shape=(), dtype=int32)
 psi:  tf.Tensor(-81513.0, shape=(), dtype=float32)
step 2.0

Iteration:  tf.Tensor(2, shape=(), dtype=int32)
 psi:  tf.Tensor(-89459.7, shape=(), dtype=float32)
step 1.0

Iteration:  tf.Tensor(3, shape=(), dtype=int32)
 psi:  tf.Tensor(-89899.1, shape=(), dtype=float32)
step 1.0

Iteration:  tf.Tensor(4, shape=(), dtype=int32)
 psi:  tf.Tensor(-89908.5, shape=(), dtype=float32)
step 0.0



In [13]:
var = ks.variance(50)
iplot([go.Scatter3d(x = X[:,0], y = X[:,1], z= var.numpy(), mode = 'markers', marker=dict(size = 2,))])